<a href="https://colab.research.google.com/github/DiprajBista/ADS-504-02-Group-7-Machine-Learning-and-Deep-Learning-for-Data-Science/blob/main/merge_nvda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Load the data
file_path = "/workspaces/ADS-504-02-Group-7-Machine-Learning-and-Deep-Learning-for-Data-Science/nvda_2018.csv"
data = pd.read_csv(file_path)

In [ ]:
# Correct column names
correct_column_names = [
    "time",
    "open",
    "high",
    "low",
    "close",
    "PlotCandle (Open)",
    "PlotCandle (High)",
    "PlotCandle (Low)",
    "PlotCandle (Close)",
    "VWAP",
    "Upper Band #1",
    "Lower Band #1",
    "Upper Band #2",
    "Lower Band #2",
    "Upper Band #3",
    "Lower Band #3",
    "MidLine",
    "ImpulseMACD",
    "ImpulseHisto",
    "ImpulseMACDSignal",
    "RSI",
    "Regular Bullish Label",
    "Regular Bullish",
    "Hidden Bullish Label",
    "Hidden Bullish",
    "Regular Bearish Label",
    "Regular Bearish",
    "Hidden Bearish Label",
    "Hidden Bearish",
]

In [ ]:
# Handle outliers
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1
data = data.clip(lower=Q1 - 1.5 * IQR, upper=Q3 + 1.5 * IQR, axis=1)

# Handle missing values
data.fillna(method="ffill", inplace=True)

# Feature Engineering
data["target"] = data["close"] - data["open"]
data["target_t+1"] = data["target"].shift(-1)
data["day_of_week"] = data["time"].apply(lambda x: pd.to_datetime(x).weekday())
data["morning_afternoon"] = data["time"].apply(
    lambda x: "morning" if pd.to_datetime(x).hour < 12 else "afternoon"
)

/tmp/ipykernel_59960/2382185470.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [ ]:
# Lag Features
for lag in range(1, 4):
    data[f"close_lag_{lag}"] = data["close"].shift(lag)

# Rolling Statistics
data["close_rolling_mean_5"] = data["close"].rolling(window=5).mean()
data["close_rolling_std_5"] = data["close"].rolling(window=5).std()

In [ ]:
# Drop unused columns
columns_to_drop = [
    "Regular Bullish",
    "Regular Bullish Label",
    "Hidden Bullish",
    "Hidden Bullish Label",
    "Regular Bearish",
    "Regular Bearish Label",
    "Hidden Bearish",
    "Hidden Bearish Label",
]
data.drop(columns=columns_to_drop, inplace=True)

# Convert categorical features to numerical
data = pd.get_dummies(
    data, columns=["day_of_week", "morning_afternoon"], drop_first=True
)

# Create the target variable: close - open
data["target"] = data["close"] - data["open"]
data["target_t+1"] = data["target"].shift(-1)

# Handle NaN introduced by the shift
data.dropna(subset=["target_t+1"], inplace=True)

# Extract features and target variable
X = data.drop(columns=["time", "close", "target", "target_t+1"])
y = data["target_t+1"]

In [ ]:
from sklearn.impute import SimpleImputer

# Ensure alignment of X and y
X = X.iloc[:-1]
y = y.iloc[:-1]

# Impute missing values
imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_imputed, y, test_size=0.2, random_state=42
)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

# Initialize models
lr_model = LinearRegression()
rf_model = RandomForestRegressor(random_state=42)
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

# Hyperparameter tuning for XGBoost
param_grid_xgb = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 6, 10],
    "learning_rate": [0.01, 0.1, 0.2],
}

random_search_xgb = RandomizedSearchCV(
    xgb_model, param_grid_xgb, n_iter=10, cv=3, scoring="r2", n_jobs=-1, random_state=42
)
random_search_xgb.fit(X_train_scaled, y_train)
best_xgb_model = random_search_xgb.best_estimator_

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import xgboost as xgb

# Assuming X_train_scaled, X_test_scaled, y_train, y_test, and random_search_xgb are already defined

# Initialize models
lr_model = LinearRegression()
best_rf_model = RandomForestRegressor(random_state=42)
best_xgb_model = xgb.XGBRegressor(**random_search_xgb.best_params_)

# Train models
lr_model.fit(X_train_scaled, y_train)
best_rf_model.fit(X_train_scaled, y_train)
best_xgb_model.fit(X_train_scaled, y_train)

# Evaluate models
lr_predictions = lr_model.predict(X_test_scaled)
rf_predictions = best_rf_model.predict(X_test_scaled)
xgb_predictions = best_xgb_model.predict(X_test_scaled)

lr_mse = mean_squared_error(y_test, lr_predictions)
lr_r2 = r2_score(y_test, lr_predictions)
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)
xgb_mse = mean_squared_error(y_test, xgb_predictions)
xgb_r2 = r2_score(y_test, xgb_predictions)

print(f"Linear Regression MSE: {lr_mse}, R2: {lr_r2}")
print(f"Random Forest MSE: {rf_mse}, R2: {rf_r2}")
print(f"XGBoost MSE: {xgb_mse}, R2: {xgb_r2}")
print(f"Best XGBoost Parameters: {random_search_xgb.best_params_}")

# Cross-Validation Scores
cv_scores_xgb = cross_val_score(
    best_xgb_model, X_train_scaled, y_train, cv=5, scoring="r2", n_jobs=-1
)
print(f"XGBoost 5-fold CV R2: {np.mean(cv_scores_xgb)}")

Linear Regression MSE: 0.030076782101481455, R2: -0.024117995763985656
Random Forest MSE: 0.03158111454399169, R2: -0.07534069375036512
XGBoost MSE: 0.02939647834490979, R2: -0.0009535722116214007
Best XGBoost Parameters: {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.01}
XGBoost 5-fold CV R2: 0.0012670863222378204
